In [8]:
import csv
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
age_train = pd.read_csv('age_train.csv',header=None)
age_train.columns = ['uId','age_group']
app_info = pd.read_csv('app_info.csv',header=None)
app_info.columns = ['appId','category']
user_basic_info = pd.read_csv('user_basic_info.csv',header=None)
user_basic_info.columns = ['uId','gender','city','prodName',
                           'ramCapacity','ramLeftRation','romCapacity',
                           'romLeftRation','color','fontSize',
                           'ct','carrier','os']
user_app_actived = pd.read_csv('user_app_actived.csv',header=None)
user_app_actived.columns = ['uId','appId']
user_behavior_info = pd.read_csv('user_behavior_info.csv',header=None)
user_behavior_info.columns = ['uId','bootTimes','AFuncTimes','BFuncTimes',
                              'CFuncTimes','DFuncTimes','EFuncTimes',
                              'FFuncTimes','FFuncSum']

In [3]:
user_basic_info = user_basic_info.dropna()
app_info = app_info.dropna()
age_train = age_train.dropna()
user_behavior_info = user_behavior_info.dropna()
user_app_actived = user_app_actived.dropna()

app_info = pd.get_dummies(app_info,columns=['category'])
del user_basic_info['city']
user_basic_info = pd.get_dummies(user_basic_info,columns=['prodName','color','ct','carrier'])
# user_basic_info = pd.get_dummies(user_basic_info,columns=['city','prodName','color','ct','carrier'])
# todo: city 和 prodName 维度太高，估计比较稀疏，要把数目少的类别归为一个other类。
# city这个特征作用不大，而且有300多维，可能可以删了
df = user_behavior_info
float_col = [each for each in df.columns if df[each].dtype=='float']
normalize_col = float_col + ['bootTimes','FFuncSum']
user_behavior_info[normalize_col] = df[normalize_col].apply(lambda x: (x - np.mean(x)) / np.std(x))

df = user_basic_info
float_col = [each for each in df.columns if df[each].dtype=='float']
user_basic_info[float_col] = df[float_col].apply(lambda x: (x - np.mean(x)) / np.std(x))

In [4]:
app_info_dict = {}
for each in app_info.values:
    app = each[0]
    cat = each[1:]
    if app in app_info.keys():
        app_info_dict[app] += cat
    else:
        app_info_dict[app] = cat
        
    
def f(x):
    x = x.split('#')
    r = []
    for each in x:
        if each in app_info_dict.keys():
            r.append(app_info_dict[each])
        else:
            r.append(np.array([0]*40))
    return sum(r)
    
user_app_actived['app_total_categery'] = user_app_actived['appId'].apply(f)

In [5]:
user_app_actived['app_numbers'] = user_app_actived['appId'].apply(lambda x:len(x.split('#')))
user_app_actived['app_average_categery'] = user_app_actived['app_total_categery'].apply(lambda x:x/sum(x))
# user_app_actived['topk_app'] = user_app_actived['app_total_categery'].apply(lambda x:x/sum(x))
# todo: 需要看app的分布直方图，选取topK或者top m 到 n。

In [6]:
temp = user_app_actived[['uId','app_average_categery']].values
temp1 = []
for i in  range(len(temp)):
    temp1.append(np.insert(temp[i][1],0,temp[i][0]))
temp1 = np.array(temp1)
temp1 = pd.DataFrame(temp1)
temp1.columns = ['uId'] + ['categery_'+str(i) for i in range(40)]
temp1['uId'] = temp1['uId'].astype('int64')
temp1[['categery_'+str(i) for i in range(40)]] = temp1[['categery_'+str(i) for i in range(40)]].astype('float64')
 
user_app_actived = pd.merge(user_app_actived,temp1, on=['uId'], how='inner')
del user_app_actived['app_total_categery']
del user_app_actived['app_average_categery']
del user_app_actived['appId']

In [20]:
user_app_usage_file = [each for each in os.listdir('./') if each.endswith('.pkl')]
user_app_usage_file.remove('dataset.pkl')
user_app_usage_file.remove('user_app_usage_features.pkl')
user_app_usage_file.sort()

temp = []
for each in user_app_usage_file:
    with open(each,'rb') as f:
        temp.append(pickle.load(f))
user_app_usage = pd.concat(temp)
user_app_usage = user_app_usage[list(user_app_usage.columns)].astype('float64')
user_app_usage = user_app_usage.dropna()

In [21]:
df = user_app_usage
float_col = [each for each in df.columns if df[each].dtype=='float']
user_app_usage[float_col] = df[float_col].apply(lambda x: (x - np.mean(x)) / np.std(x))
user_app_usage = user_app_usage.reset_index()
user_app_usage = user_app_usage.rename(columns={'':'uId'})

In [24]:
user_app_usage.head()

,uId,weekdayDuration_0,weekdayDuration_1,weekdayDuration_2,weekdayDuration_3,weekdayDuration_4,weekdayDuration_5,weekdayDuration_6,weekdayDuration_7,weekdayDuration_8,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
0,1000321,-0.604203,-0.080507,0.035063,-0.421375,-0.433713,-0.268794,-0.219260,-0.275192,-0.293632,...,-0.118505,0.637197,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,0.037983,-0.163153,0.003405
1,1000364,0.629147,-0.431867,0.607126,0.935634,-0.433713,-0.268794,0.241979,-0.275192,-0.293632,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,-0.712038,-0.644853,-0.029273
2,1000387,0.091165,-0.003100,-0.312346,0.963125,-0.433713,0.072664,-0.219260,-0.275192,-0.293632,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,0.698324,1.185658,-0.042505
3,1000395,-0.623263,-0.361527,-0.294413,-0.406289,-0.433713,-0.268794,-0.219260,-0.275192,0.164485,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,1.017208,-0.633911,-0.315957,-0.058802
4,1000466,-0.604031,1.650210,-0.036454,0.283342,-0.433713,-0.268794,-0.193845,-0.160322,-0.285574,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,0.417066,0.398350,-0.018566


In [12]:
user_basic_info.head()

,uId,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,fontSize,os,prodName_p001,prodName_p0010,...,ct_2g#wifi,ct_3g,ct_3g#wifi,ct_4g,ct_4g#wifi,ct_wifi,carrier_China_Mobile,carrier_China_Telecom,carrier_China_Unicom,carrier_othercp
0,1000001,0,-1.462255,1.075414,-1.045085,0.139472,0.755730,-0.139578,0,0,...,0,0,0,1,0,0,1,0,0,0
2,1000004,1,-1.462255,0.316360,-1.045085,-1.312003,-0.541048,-0.139578,0,0,...,0,0,0,0,1,0,1,0,0,0
4,1000007,1,0.791664,-1.418619,0.998926,0.296388,-0.541048,1.097250,0,0,...,0,0,0,0,1,0,0,1,0,0
5,1000008,1,0.791664,0.316360,0.823725,0.178701,0.755730,-0.139578,0,0,...,0,0,0,0,1,0,0,1,0,0
8,1000012,0,-0.710949,-0.225821,-0.422148,-0.135132,2.052594,-0.139578,0,0,...,0,0,0,0,0,1,1,0,0,0


1. user_behavior_info,  
2. user_basic_info,  
3. user_app_actived,  
4. user_app_usage

In [32]:
join_data = age_train
print(len(join_data))
join_data = pd.merge(join_data,user_behavior_info, on=['uId'], how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_basic_info,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_actived,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_usage,on=['uId'],how='inner')
print(len(join_data))
join_data.dropna()
print(len(join_data))

train_data = join_data
train_data = optimize_mem(train_data)

2010000
2010000
1370640
1370640
1345506
1345506


In [56]:
len(train_data.dropna())

1345505

In [50]:
with open('train.data','wb') as f:
    pickle.dump(train_data,f)

In [38]:
age_test = pd.read_csv('age_test.csv',header=None)
age_test.columns = ['uId']

In [39]:
join_data = age_test
print(len(join_data))
join_data = pd.merge(join_data,user_behavior_info, on=['uId'], how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_basic_info,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_actived,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_usage,on=['uId'],how='inner')
print(len(join_data))
join_data.dropna()
print(len(join_data))

test_data = join_data
test_data = optimize_mem(test_data)

502500
502500
342926
342926
336756
336756


In [41]:
with open('test.data','wb') as f:
    pickle.dump(test_data,f)

In [42]:
test_data.head()

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,gender,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
0,1000008,-0.479404,0.093857,-0.048000,-0.183851,0.072881,0.039975,-0.167348,0.194304,1,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,1.501626,-0.117324,-0.231183,-0.004815
1,1000017,0.164397,-0.524734,-0.109986,-0.183851,-0.230666,-0.229489,-0.167348,-0.134224,1,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,-0.450193,-0.274617,-0.039494
2,1000030,-0.050203,-0.524734,-0.083420,-0.183851,-0.166921,-0.203984,-0.167348,0.374811,0,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,-0.326705,-0.286914,-0.022809
3,1000031,0.557831,0.300054,-0.109986,-0.205312,-0.220547,-0.221727,-0.167348,-0.088951,0,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,-0.442617,-0.594877,0.015087
4,1000039,-0.479404,0.712448,0.217656,-0.044357,0.065799,0.317201,3.693370,-0.134224,1,...,-0.118505,-0.050118,-0.09784,-0.087272,-0.080386,-0.079847,-0.090034,0.317631,0.476060,-0.030231


In [29]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # 我们假设这不是一个df，而是一个 Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # 将 bytes 转化成 megabytes
    return "{:03.2f} MB".format(usage_mb)

def optimize_mem(sll):
    print(mem_usage(sll))
    sll_int = sll.select_dtypes(include=['int']) # 用 DataFrame.select_dtypes 来选中表中的 int数据
    converted_int = sll_int.apply(pd.to_numeric,downcast='unsigned') # 用pd.to_numeric()来降低我们的数据类型

    compare_ints = pd.concat([sll_int.dtypes,converted_int.dtypes],axis=1)
    compare_ints.columns = ['before','after']
    
    sll_float = sll.select_dtypes(include=['float'])
    converted_float = sll_float.apply(pd.to_numeric,downcast='float')

    compare_floats = pd.concat([sll_float.dtypes,converted_float.dtypes],axis=1)
    compare_floats.columns = ['before','after']
    
    sll[converted_int.columns] = converted_int
    sll[converted_float.columns] = converted_float

    print(mem_usage(sll))
    return sll